In [1]:
import requests
import os
import errno
import zipfile
import csv
import urllib
import glob
import pandas as pd

# Constants
USERNAME = 'cai.li1@husky.neu.edu'
PASSWORD = 'Hf9tN]tm'
START = 'Q12001'
END = 'Q12011'
DIR_NAME = "data/"
login_page_url = 'https://freddiemac.embs.com/FLoan/secure/auth.php'
download_page_url = 'https://freddiemac.embs.com/FLoan/Data/download2.php'
REMOVE_UNZIPPED_FILES = False
VERBOSE_MODE=True

FIELDS = 'ABCDEFGHIJKL0MNOPQRSTUVWXY'

## Directory creation if doesn't exist
def create_directory(dir_name):
    os.getcwd()
    if not os.path.exists(dir_name):
        try:
            os.makedirs(dir_name)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise

## gets the zipped files and extrcts the contents into unzipped folder
def get_data_from_url(quarter):
    print('downloading...')
    urllib.request.urlretrieve('https://freddiemac.embs.com/FLoan/Data/historical_data1_' + str(quarter) + '.zip',
                               DIR_NAME + str(quarter) + '.zip')
    # unzip_files(year)
    try:
        zip_ref = zipfile.ZipFile(DIR_NAME + str(quarter) + '.zip', 'r')
        zip_ref.extractall(DIR_NAME)
        zip_ref.close()
        
        write_into_csv(quarter)
    except zipfile.BadZipfile:
        print (zipfile.BadZipfile)

## Creates a consolidated file

## Removes files form the directory
def clean_directory(quarter):
    os.remove(DIR_NAME + str(quarter) + '.zip')
    os.remove(DIR_NAME + "historical_data1_" + str(quarter) + '.txt')
    os.remove(DIR_NAME + "historical_data1_time_" + str(quarter) + '.txt')

## Creates a consolidated file
def create_csv():
    
    fields = 'ABCDEFGHIJKL0MNOPQRSTUVWX'
    with open(DIR_NAME + '/joinedRawData.csv', 'w') as file:
        writer = csv.writer(file)
        writer.writerows([fields])
        file.close()
    return DIR_NAME + '/joinedRawData.csv'
    
# Writes the text file lines into csv
def write_into_csv(quarter):
    with open(DIR_NAME + "historical_data1_" + str(quarter) + '.txt', 'r') as sourceFile:
        sourcelines = sourceFile.read()
        sourcelines = sourcelines.replace(",", "_")
        sourcelines = sourcelines.replace("|", ",")
        sourcelines = sourcelines.replace(",\n", "\n")
        
#        with open(DIR_NAME + '/joinedRawData.csv', 'a') as destinationFile:
#            destinationFile.write(sourcelines)
        
        with open(DIR_NAME + str(quarter) +'.csv', 'w') as destinationFile:
            writer = csv.writer(destinationFile)
            writer.writerows([FIELDS])
            destinationFile.write(sourcelines)
    print('written to csv.')
    clean_directory(quarter)
    
def data_cleaning():
    fileList = fileList = glob.glob('data/*.csv')
    for file in fileList:
        df = pd.read_csv(file,error_bad_lines=False)
        print('categorical cleaning...')
        for col in ['C','G','H','M','N','O','P','Q','T','U','V','W','X']:
            mode = pd.DataFrame(df.groupby(col).size().rename('cnt')).idxmax()[0]
            df[col] = df[col].fillna(mode)
        print('numerical cleaning...')
        for col in ['E','F','I','J','K','L','R']:
            dfmean = df[(df[col] != 999)|(df[col] != None)]
            mean = int(dfmean[col].mean(axis=0))
            df[col] = df[col].fillna(mean)
        
        filename = "%sclean.csv"%file[:-4]
        df.to_csv(filename)
        
        print('cleaning finished.')
        
        
def preprocessing():
    fileList = fileList = glob.glob('data/*clean.csv')
    cat_columns = ['C','G','H','M','N','O','P','Q','T','U','V','W','X']
    num_columns = ['A','B','D','E','F','I','J','K','L','R','0']
    for file in fileList:
        df = pd.read_csv(file,error_bad_lines=False)
        
        cat = pd.get_dummies(df[cat_columns])
        df = pd.concat([df[num_columns],cat],axis = 1).fillna(0)
        
        filename = "%sprocessed.csv"%file[:-9]
        df.to_csv(filename)
        
        print('processing finished.')
        
## Main Program Execution
def start_execution():
    with requests.Session() as sess:
        sess.get(login_page_url);
        php_session_cookie = sess.cookies['PHPSESSID']
        login_payload = {'username' : USERNAME, 'password' : PASSWORD,'cookie':php_session_cookie}
        sess.post(login_page_url, data = login_payload)
        download_page_payload = {'accept': 'Yes', 'action': 'acceptTandC', 'acceptSubmit': 'Continue', 'cookie': php_session_cookie}
        sess.post(download_page_url, data=download_page_payload)
        create_directory(DIR_NAME)
        #create_csv()
        #download files
        
        #data_cleaning()
        #preprocessing()

## Calling all the main functions
if __name__ == "__main__":
    start_execution()
    print('finished.')



finished.


In [2]:
#Use downloaddata.py with proper settings to download the data.

import pandas as pd
df1 = pd.read_csv('data/Q12001processed.csv')
df2 = pd.read_csv('data/Q12002processed.csv')
df3 = pd.read_csv('data/Q12003processed.csv')
df4 = pd.read_csv('data/Q12004processed.csv')
df5 = pd.read_csv('data/Q42005processed.csv')

df6 = pd.read_csv('data/Q12006processed.csv')
df7 = pd.read_csv('data/Q12007processed.csv')
df8 = pd.read_csv('data/Q12008processed.csv')
df9 = pd.read_csv('data/Q12009processed.csv')
df10 = pd.read_csv('data/Q12010processed.csv')
df11 = pd.read_csv('data/Q12011processed.csv')
df12 = pd.read_csv('data/Q12012processed.csv')

df72 = pd.read_csv('data/Q22007processed.csv')
df73 = pd.read_csv('data/Q32007processed.csv')
df74 = pd.read_csv('data/Q42007processed.csv')

In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def printPerformance(pred,y_test):
    print(pred)
    print("RMSE: %.2f"
          % sqrt(mean_squared_error(y_test, pred)))
    print("MAPE: %.2f"
          % mean_absolute_percentage_error(y_test, pred)+'%')
    print("MAE: %.2f"
          % mean_absolute_error(y_test, pred))

In [4]:
#What if there is a financial crisis...
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(0.3)
selector2 = VarianceThreshold(0.2)
selector3 = VarianceThreshold(0.19)

In [5]:
df_train = pd.concat([df7,df72,df73,df74],axis = 0).fillna(0)
df_test = pd.concat([df8,df72,df73,df74],axis = 0).fillna(0)

#Pipline random forest
from sklearn.ensemble import RandomForestRegressor
randomForest = RandomForestRegressor(max_depth= 18, n_estimators = 16, random_state=2)

y_train1 = df_train['0']
X_train1 = selector.fit_transform(df_train.drop(['0'], axis = 1))

y_test1 = df_test['0']
X_test1 = selector.fit_transform(df_test.drop(['0'], axis = 1))

print('fitting...')
randomForest.fit(X_train1,y_train1)
print('fitted.')
printPerformance(randomForest.predict(X_test1),y_test1)

#Two Years Later..
print('Two Years Later..')
y_test11 = df9['0']
X_test11 = selector.fit_transform(df9)
randomForest.fit(X_train1,y_train1)
printPerformance(randomForest.predict(X_test11),y_test11)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


fitting...
fitted.
[6.06318916 6.4171998  6.30951532 ... 6.28083239 6.30371094 5.0078125 ]
RMSE: 0.35
MAPE: 4.32%
MAE: 0.26
Two Years Later..
[7.0546875 7.3203125 7.359375  ... 6.96875   7.0234375 6.96875  ]
RMSE: 2.08
MAPE: 42.23%
MAE: 2.05


In [ ]:
#What if there is a economy boom...
df_train2 = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10],axis = 0).fillna(0)

In [ ]:
#What if there is a economy boom...
df_train2 = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10],axis = 0).fillna(0)
y_train2 = df_train2['0']
X_train2 = selector2.fit_transform(df_train2.drop(['0'], axis = 1))

y_test2 = df12['0']
X_test2 = selector3.fit_transform(df12.drop(['0'], axis = 1))
print(X_train2.shape)
print('fitting...')
randomForest.fit(X_train2,y_train2)
print('fitted.')
printPerformance(randomForest.predict(X_test2), y_test2)

In [9]:
y_test2 = df12['0']
X_test2 = selector3.fit_transform(df12.drop(['0'], axis = 1))

printPerformance(randomForest.predict(X_test2), y_test2)

[7.46529516 7.20356593 7.14046844 ... 7.71617187 7.65429688 7.48874423]
RMSE: 3.64
MAPE: 98.38%
MAE: 3.57


In [11]:
#What if there is a regime change from election
y_train3 = df6['0']
X_train3 = selector3.fit_transform(df6.drop(['0'],axis = 1))
print(X_train3.shape)
y_test3 = y_test2
X_test3 = X_test2
print('fitting...')
randomForest.fit(X_train3,y_train3)
printPerformance(randomForest.predict(X_test3),y_test3)

(313491, 19)
fitting...
[6.2892583  6.3452474  6.29616552 ... 5.82299758 5.92477492 5.94840169]
RMSE: 2.32
MAPE: 62.49%
MAE: 2.28
